In [85]:
import os
! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all firecrawl-py PyMuPDF sentence_transformers python-dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [86]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
## index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.docstore.document import Document

# Load PDF documents
pdf_file_paths = ["./data/Medical Policy FY 23-24.pdf", "./data/Provident fund policy.pdf"]

docs_list = []
for pdf_path in pdf_file_paths:
    loader = PyMuPDFLoader(pdf_path)
    try:
        loaded_docs = loader.load()
        docs_list.extend(loaded_docs)
    except Exception as e:
        print(f"Error loading {pdf_path}: {e}")

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=250, chunk_overlap=0)
doc_splitter = text_splitter.split_documents(docs_list)

# Filter and clean metadata
filtered_doc = []
for doc in doc_splitter:
    if isinstance(doc, Document) and hasattr(doc, 'metadata'):
        clean_metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
        filtered_doc.append(Document(page_content=doc.page_content, metadata=clean_metadata))

# Add to vectorDB
embedding = GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2.gguf2.f16.gguf", gpt4all_kwargs={'allow_download': 'True'})
vectorstore = Chroma.from_documents(
    documents=filtered_doc,
    collection_name="rag-chroma",
    embedding=embedding,
)

retriever = vectorstore.as_retriever()

In [8]:
# local_llm = 'gemma'
local_llm = 'llama3'
# local_llm = 'mistral'

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# Load a pre-trained model for semantic similarity
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Define prompt templates
context_prompt = PromptTemplate(
    template="""system You are an assistant for question-answering tasks.
    Use the following context to answer the question. Avoid phrases like "Based on the provided context". explain the answer in the end. and make a heading with paragraph
    Question: {question}
    Context: {context}
    Answer: assistant""",
    input_variables=["question", "context"],
)

no_context_prompt = PromptTemplate(
    template="""system You are an assistant for question-answering tasks.
    Answer the question concisely and directly.
    Question: {question}
    Answer: assistant""",
    input_variables=["question"],
)

llm = ChatOllama(model=local_llm, temperature=0)

# Load a text classification model for future usefulness prediction
usefulness_classifier = pipeline("text-classification", model="bert-base-uncased")


# Function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Enhanced Function to check relevance using semantic similarity
def is_relevant(question, context_docs, threshold=0.3):
    context_text = format_docs(context_docs)

    # Split context into smaller chunks
    context_chunks = context_text.split('\n\n')

    # Encode the question
    question_embedding = model.encode(question, convert_to_tensor=True)

    max_similarity = 0
    for chunk in context_chunks:
        # Encode each chunk of context
        context_embedding = model.encode(chunk, convert_to_tensor=True)

        # Compute similarity
        similarity = util.pytorch_cos_sim(question_embedding, context_embedding)
        max_similarity = max(max_similarity, similarity.item())

    # Return True if similarity is above threshold  
    return max_similarity > threshold


# Function to determine if a question/statement is useful for the future
def is_useful_for_future(question):
    # Use the classification pipeline to predict usefulness
    result = usefulness_classifier(question)
    is_useful = result[0]['label'] == 'USEFUL'
    if is_useful:
        with open("useful_question.txt", "a") as file:
            file.write(question + "\n")

    return result[0]['label'] == 'IMPORTANT'


# Function to get the appropriate chain based on relevance
def get_chain(question, context_docs):
    if is_relevant(question, context_docs):
        print("Relevant")
        return context_prompt | llm | StrOutputParser()
    else:
        print("Irrelevant")
        return no_context_prompt | llm | StrOutputParser()


# Main function to run the question-answering process
def answer_question(question):
    docs = retriever.invoke(question)
    rag_chain = get_chain(question, docs)
    if is_relevant(question, docs):
        generation = rag_chain.invoke({"context": docs, "question": question})
    else:
        generation = rag_chain.invoke({"question": question})

    # Check if the question is_useful useful for the future
    if is_useful_for_future(question):
        print(f"This question/statement is useful for the future.")
    else:
        print(f"This question/statement is not particularly useful for the future.")

    return generation


# Example question
# question = "What is the vesting period for employer contributions to the provident fund?"
question = "My dog's illness can be cured by Antihistamines medicines."

answer = answer_question(question)
print(answer)


In [89]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from sentence_transformers import SentenceTransformer, util
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationSummaryMemory

# Load a pre-trained model for semantic similarity
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

llm = ChatOllama(model=local_llm, temperature=0)

# Define memory

# Function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Enhanced Function to check relevance using semantic similarity
def is_relevant(question, context_docs, threshold=0.3):
    context_text = format_docs(context_docs)
    context_chunks = context_text.split('\n\n')
    question_embedding = model.encode(question, convert_to_tensor=True)
    max_similarity = 0
    for chunk in context_chunks:
        context_embedding = model.encode(chunk, convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(question_embedding, context_embedding)
        max_similarity = max(max_similarity, similarity.item())
    return max_similarity > threshold


# Function to get the appropriate chain based on relevance
def get_chain(question, context_docs):
    if is_relevant(question, context_docs):
        print("Relevant")

        template = """system You are an assistant for question-answering tasks.
        Use the following context to answer the question. Avoid phrases like "Based on the provided context". 
        Explain the answer in the end and make a heading with paragraph.
        Question: {question}
        Context: {context}
        Answer: assistant"""

        prompt = PromptTemplate(input_variables=["question", "context"], template=template)

        return ConversationChain(
            prompt=prompt,
            llm=llm,
            verbose=True,
            memory=ConversationBufferMemory(),
        )
    else:
        print("Irrelevant")

        template = """system You are an assistant for question-answering tasks.
        Answer the question concisely and directly.
        Question: {question}
        Answer: assistant"""

        prompt = PromptTemplate(input_variables=["question", "chat_history"], template=template)

        return ConversationChain(
            prompt=prompt,
            llm=llm,
            verbose=True,
            memory=ConversationSummaryMemory(llm=llm),
        )


# Main function to run the question-answering process
def answer_question(question):
    docs = retriever.invoke(question)
    rag_chain = get_chain(question, docs)

    if is_relevant(question, docs):

        generation = rag_chain.invoke({"context": docs, "question": question})
    else:
        generation = rag_chain.invoke({"question": question})

    return generation


# Example question
question = "What is the vesting period for employer contributions to the provident fund?"
answer = answer_question(question)
print(answer)


Relevant


ValidationError: 1 validation error for ConversationChain
__root__
  Got unexpected prompt input variables. The prompt expects ['context', 'question'], but got ['history'] as inputs from memory, and input as the normal input key. (type=value_error)

In [9]:
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain_community.chat_models import ChatOllama
from langchain.chains import LLMChain, ConversationChain


llm = ChatOllama(model=local_llm, temperature=0)

In [10]:
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [11]:
conversation("what is my name?")



/Users/harisali/development/codeBase/langchain-lamaindex/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'input': 'what is my name?',
 'history': '',
 'response': "I'm happy to help! Unfortunately, I don't have that information stored in our current conversation or any previous interactions we may have had. As a conversational AI, I don't have the ability to access external databases or personal information about individuals without explicit consent. My training data only includes general knowledge and doesn't contain specific details about individual humans. If you'd like to share your name with me, I'm happy to chat with you!"}

In [ ]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [ ]:
count_tokens(
    conversation,
    "what is my name?"
)

In [ ]:
print(conversation.memory.buffer)



In [15]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversationSummary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=llm)
)

In [16]:
print(conversationSummary.memory.prompt.template)


Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [17]:
count_tokens(
    conversationSummary,
    "Which data source types could be used to give context to the model?"
)

NameError: name 'count_tokens' is not defined

In [ ]:
print(conversationSummary.memory.buffer)

